# ACS Gini Ratio, Table B19083

In [1]:
import pandas as pd
import glob
import csv

### Define a function to process a single year release

In [2]:
def clean_dataset(path):
    df = pd.read_csv(path, skiprows=1).rename(columns={
        'id': 'FIPS',
        'Geographic Area Name': 'Town',
        'Estimate!!Gini Index': 'Gini Ratio',
        'Margin of Error!!Gini Index': 'Margins of Error'
    })
    
    # Remove nonsense geographies
    df = df[ ~df.Town.str.contains('not defined')]
    
    # Clean up FIPS code and Town name
    df.FIPS = df.FIPS.apply(lambda x: x.split('US')[1])
    df.Town = df.Town.apply(lambda x: x.split(' town, ')[0])
    
    # Guess release year from the file name
    year = int(path.split('.B19083')[0].split('ACSDT5Y')[1])
    df['Year'] = '{}-{}'.format(year-4, year)
    
    return df

### Read all files from `raw/` folder and compile into one final dataset

In [3]:
paths = glob.glob('raw/*B19083*.csv')    

# Put all years into a single data frame
gini_wide = pd.concat(map(clean_dataset, paths))
gini_wide['Measure Type'] = 'Number'

# Transform wide to long
gini = pd.melt(
    gini_wide,
    id_vars=['Town', 'FIPS', 'Year', 'Measure Type'],
    value_vars=['Margins of Error', 'Gini Ratio'],
    var_name='Variable',
    value_name='Value',
)

gini.Value = gini.Value.apply(float)

# Sort by town/year and save
gini.sort_values(['Town', 'Year']).to_csv('data/gini-ratio-2019.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)